In [1]:
import psycopg2 as pg
import time
from io import StringIO
import pandas as pd
import os
from datetime import datetime
from datetime import date
import numpy as np
from tqdm.notebook import tqdm
import calendar
import math
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm
import re

In [2]:
###################################################### INPUTS ################################################################

index = 'Nifty'                             # Format : 'BankNifty' / 'Nifty' / 'FinNifty'
schema = 'WeeklyI'                          # Format : 'MonthlyI' / 'WeeklyII' / 'QuarterlyIII' / 'HalfYearlyI' / 'YearlyIV'
timeframe = '1'                             # Format : '1/5/15' or 'E'
one_file = 'N'                              # Format : 'Y' if Opt+Eq data in one file, 'N' if two different files
if one_file == 'N':
    # path of options file
    opt_file_input_path = r"C:\Users\admin\Desktop\Nifty_WeeklyI_Data\Nifty_WeeklyI_Opt_1min.csv"

    #path of equity file
    eq_file_input_path = r"C:\Users\admin\Desktop\Nifty_EqData\Nifty_1min.csv"

else:
    # path where your file is present (Opt+Eq)
    file_path = ''

################################################## CODE STARTS FROM HERE #####################################################
if timeframe!= 'E':
    # Defining greeks function
    def calculate_greeks_vectorized(spot, strike, time_to_expiry, option_price, option_type):
        norm_dist = norm.cdf
        rate = 0.065
        dividend = 0.015
        is_call_option = np.where(option_type == 'CE', True, False)
        is_put_option = np.where(option_type == 'PE', True, False)
        target_option = np.where(is_call_option, option_price, 0)
        upper = np.where(is_call_option, 5.0, 0.1)
        lower = np.where(is_call_option, 0.0, 0.0)
        while np.any((upper - lower) > 0.00001):
            d_one = (np.log(spot / strike) + ((rate - dividend + (0.5 * (((upper + lower) / 2) ** 2))) * time_to_expiry)) / (((upper + lower) / 2) * (time_to_expiry ** 0.5))
            d_two = d_one - ((upper + lower) / 2) * time_to_expiry ** 0.5
            nd_one = norm_dist(d_one)
            nd_two = norm_dist(d_two)
            call_premium = (np.exp(-dividend * time_to_expiry) * (spot * nd_one)) - (strike * np.exp(-rate * time_to_expiry) * nd_two)
            mask = call_premium > target_option
            upper = np.where(mask, (upper + lower) / 2, upper)
            lower = np.where(mask, lower, (upper + lower) / 2)
        calldelta1=np.where(is_call_option,(nd_one) * np.exp(-dividend * time_to_expiry),0)
        call_delta=np.where((is_call_option)&(time_to_expiry==0)&(spot > strike), 1.0, calldelta1)
        call_iv = np.where(is_call_option, (upper + lower) / 2, 0.0)
        ndash_one = 1 / (2 * np.pi) ** 0.5 * (np.exp(-(d_one ** 2) / 2))
        call_vega = np.where(is_call_option, spot * ((time_to_expiry ** (1 / 2)) * ndash_one * np.exp(-dividend * time_to_expiry)) / 100, 0.0)
        call_gamma = np.where(is_call_option, ((((1 / np.sqrt((2 * np.pi))) * np.exp(((-1 * (d_one ** 2)) / 2))) * np.exp(((-1 * time_to_expiry) * dividend))) / ((spot * call_iv) * np.sqrt(time_to_expiry))), 0.0)
        call_theta =np.where(is_call_option,((((-1 * ((((spot * ((1 / np.sqrt((2 * np.pi))) * np.exp(((-1 * (d_one ** 2)) / 2)))) * call_iv) * np.exp(((-1 * time_to_expiry) * dividend))) / (2 * np.sqrt(time_to_expiry)))) + ((dividend * spot) * call_delta)) - (((rate * strike) * np.exp(((-1 * rate) * time_to_expiry))) * norm_dist(d_two)))) / 365,0)
        target_option1 = np.where(is_put_option, option_price, 0)
        upper1 = np.where(is_put_option, 5.0, 0.1)
        lower1 = np.where(is_put_option, 0.0, 0.0)
        while np.any((upper1 - lower1) > 0.00001):
            d_one1 = (np.log(spot / strike) + ((rate - dividend + (0.5 * (((upper1 + lower1) / 2) ** 2))) * time_to_expiry)) / (((upper1 + lower1) / 2) * (time_to_expiry ** 0.5))
            d_two1 = d_one1 - ((upper1 + lower1) / 2) * time_to_expiry ** 0.5
            nd_one1 = norm_dist(d_one1)
            nd_two1 = norm_dist(d_two1)
            put_premium =  np.exp(-rate * time_to_expiry) * strike * (norm_dist(-d_two1)) -np.exp(-dividend * time_to_expiry) *  spot* (norm_dist( -d_one1))
            mask1 = put_premium > target_option1
            upper1 = np.where(mask1, (upper1 + lower1) / 2, upper1)
            lower1 = np.where(mask1, lower1, (upper1 + lower1) / 2)
        put_delta1 = np.where(is_put_option, (nd_one1 - 1) * np.exp(-dividend * time_to_expiry), 0.0)
        put_delta=np.where((is_put_option)&(time_to_expiry==0)&(spot > strike), 1.0, put_delta1)
        put_iv = np.where(is_put_option, (upper1 + lower1) / 2, 0.0)
        ndash_one1 = 1 / (2 * np.pi) ** 0.5 * (np.exp(-(d_one1 ** 2) / 2))
        put_vega = np.where(is_put_option, spot * ((time_to_expiry ** (1 / 2)) * ndash_one1 * np.exp(-dividend * time_to_expiry)) / 100, 0.0)
        put_gamma = np.where(is_put_option, ((((1 / np.sqrt((2 * np.pi))) * np.exp(((-1 * (d_one1 ** 2)) / 2))) * np.exp(((-1 * time_to_expiry) * dividend))) / ((spot * put_iv) * np.sqrt(time_to_expiry))), 0.0)
        put_theta =  np.where(is_put_option,(((((-1 * ((((spot * ((1 / np.sqrt((2 * np.pi))) * np.exp(((-1 * d_one1 ** 2) / 2)))) * put_iv) * np.exp(((-1 * time_to_expiry) * dividend)))))/ (2 * np.sqrt(time_to_expiry))) - (((dividend * spot) * norm_dist((-1 * d_one1))) * np.exp(((-1 * time_to_expiry) * dividend)))) + (((rate * strike) * np.exp(((-1 * rate) * time_to_expiry))) * norm_dist((-1 * d_two1))))) / 365,0.0)
        iv= np.where(is_call_option,call_iv,put_iv)
        delta= np.where(is_call_option,call_delta,put_delta)
        gamma= np.where(is_call_option,call_gamma,put_gamma)
        theta= np.where(is_call_option,call_theta,put_theta)
        vega=np.where(is_call_option,call_vega,put_vega)
        return iv, delta, gamma, theta, vega
        
    time1 = datetime.strptime('15:29:59', '%H:%M:%S').time()
    st_g = time.time()
    
    if one_file == 'N':
        
        ################## EQUITY DATA
        eq_df = pd.read_csv(eq_file_input_path)
        eq_df = eq_df.rename(columns={' Time' : 'Time','Open' : 'EQ_Open','High' : 'EQ_High','Low' : 'EQ_Low','Close' : 'EQ_Close'})
        eq_df = eq_df[['Date', 'Time', 'EQ_Open', 'EQ_High', 'EQ_Low', 'EQ_Close']]
        eq_df['Date'] = pd.to_datetime(eq_df['Date'], format = 'mixed',dayfirst=True)
        eq_df['Time'] = pd.to_datetime(eq_df['Time']).dt.time
            
        ################## OPTIONS DATA
        opt_df = pd.read_csv(opt_file_input_path)
        opt_df = opt_df.rename(columns={'date' : 'Date','ticker' : 'Ticker','time' : 'Time','open' : 'Open','high' : 'High','low' : 'Low','close' : 'Close','volume' : 'Volume','Open Int' : 'OpenInterest'})
        opt_df = opt_df.rename(columns={'Open' : 'Adj_Open','High' : 'Adj_High','Low' : 'Adj_Low','Close' : 'Adj_Close'})
        opt_df['Date'] = pd.to_datetime(opt_df['Date'], dayfirst=True , format = '%Y-%m-%d')
        opt_df['Year'] = opt_df['Date'].dt.year
        opt_df = opt_df.drop(['Year'], axis=1)
        opt_df['Time'] = pd.to_datetime(opt_df['Time']).dt.time
        opt_df = opt_df[opt_df['Time'] <= time1]
        opt_df = opt_df.merge(eq_df, on=['Date', 'Time'], how='left')
    
    else:
        opt_df = pd.read_csv(file_path)
        
    ################# GETTING THE EXPIRY SHEET FROM SQL
    if index != 'FinNifty':
        if index=='BankNifty' and schema[:schema.find('y')+1]=='Weekly':
            engine = pg.connect("dbname='ExpiryDates' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
            exp_df = pd.read_sql(f'select * from "public"."{index}{schema[:schema.find("y")+1]}Expiry"', con=engine)
            engine.close()
        else:
            engine = pg.connect("dbname='ExpiryDates' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
            exp_df = pd.read_sql(f'select * from "public"."{schema[:schema.find("y")+1]}Expiry"', con=engine)
            engine.close()
    elif index == 'FinNifty':
        engine = pg.connect("dbname='ExpiryDates' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
        exp_df = pd.read_sql(f'select * from "public"."{index}{schema[:schema.find("y")+1]}Expiry"', con=engine)
        exp_df.rename(columns={'date':'Date'},inplace=True)
        engine.close()
    exp_df['Date'] = pd.to_datetime(exp_df['Date'], format='mixed',dayfirst=True)
    
    if schema[schema.find('y')+1:] == 'I':
        opt_df = opt_df.merge(exp_df[['Date', 'E1']], on=['Date'], how='left')
        opt_df = opt_df.rename(columns={'E1' : 'Expiry_Date'})
    elif schema[schema.find('y')+1:] == 'II':
        opt_df = opt_df.merge(exp_df[['Date', 'E2']], on=['Date'], how='left')
        opt_df = opt_df.rename(columns={'E2' : 'Expiry_Date'})
    elif schema[schema.find('y')+1:] == 'III':
        opt_df = opt_df.merge(exp_df[['Date', 'E3']], on=['Date'], how='left')
        opt_df = opt_df.rename(columns={'E3' : 'Expiry_Date'})
    opt_df['Date'] = opt_df['Date'].astype(str)
    opt_df['Time'] = opt_df['Time'].astype(str)
    opt_df['Expiry_Date'] = opt_df['Expiry_Date'].astype(str)
    opt_df['Datetime'] = pd.to_datetime(opt_df['Date'] + ' ' + opt_df['Time'], dayfirst=True,format = '%Y-%m-%d %H:%M:%S')
    opt_df['Expiry_Datetime'] = pd.to_datetime(opt_df['Expiry_Date'] + ' ' + '15:30:00', format = 'mixed',dayfirst=True)
    opt_df['MTE'] = opt_df['Expiry_Datetime'] - opt_df['Datetime']
    opt_df['MTE'] = opt_df['MTE'].dt.total_seconds()/60
    opt_df['MTE'] = opt_df['MTE']/(365 * 24 * 60)
    opt_df['Final_strike'] = opt_df['Ticker'].str.replace(f'{index.upper()}{schema[:schema.find("y")+1].upper()}-{schema[schema.find("y")+1:]}',"").str[:-2].astype(int)
    opt_df['Option_Type'] = opt_df['Ticker'].str[-2:]
    
    greeks=calculate_greeks_vectorized(opt_df['EQ_Close'], opt_df['Final_strike'], opt_df['MTE'], opt_df['Adj_Close'], opt_df['Option_Type'])
    opt_df['IV'], opt_df['Delta'], opt_df['Gamma'], opt_df['Theta'], opt_df['Vega'] = greeks[0], greeks[1], greeks[2], greeks[3], greeks[4]
    opt_df = opt_df.sort_values(by=['Date', 'Time', 'Option_Type', 'Final_strike'])
    opt_df.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close'},inplace=True)
    opt_df = opt_df[['Ticker','Date', 'Time', 'Open','High','Low','Close','Volume','Open_Interest','EQ_Open','EQ_High','EQ_Low','EQ_Close','Expiry_Date','IV', 'Delta', 'Gamma', 'Theta', 'Vega',"Option_Type",'Final_strike']]
    opt_df=opt_df.reset_index(drop=True)
    opt_df = opt_df.drop_duplicates()
    opt_df.to_csv(rf'C:\\users\\admin\\desktop\\{index}_{schema}_Greeks.csv',index=False)
    print("\nGreeks Generated!")
    et_g = time.time()
    print("Time taken to generate greeks",et_g-st_g)

else:
    def greeks(Spot, Strike, MTE, OPT, Option_Type):
        if Option_Type == 'CE':
            NormDist = norm.cdf
            Rate = 0.065
            Dividend = 0.015
            Upper = 5
            Lower = 0
            TargetCE = OPT
            if MTE == 0:
                CallIV = 5
                CallDelta = 1 if Spot > Strike else 0
                CallGamma = 0
                CallTheta = 0
                CallVega = 0
            else:
                while ((Upper - Lower) > 0.00001):   
                    dOne = (math.log(Spot/Strike)+((Rate - Dividend + (0.5 * (((Upper + Lower)/2)**2)))*MTE))/(((Upper + Lower)/2)*(MTE**0.5))
                    dTwo = dOne - ((Upper + Lower)/2) * MTE ** 0.5
                    ndOne = NormDist(dOne)
                    ndTwo = NormDist(dTwo)
                    CallPremium =  (math.exp(-Dividend * MTE) * (Spot * ndOne)) - (Strike * math.exp(-Rate * MTE) * ndTwo)
                    if(CallPremium > TargetCE):
                        Upper = (Upper + Lower) / 2
                    else:
                        Lower = (Upper + Lower) / 2
                CallIV = (Upper + Lower) / 2  
                ndashOne = 1 / (2 * math.pi) ** 0.5 * (math.exp(-(dOne**2)/2))
                CallVega = Spot*((MTE**(1/2))*ndashOne*math.exp(-Dividend*MTE))/100
                CallGamma = ((((1/math.sqrt((2*math.pi)))*math.exp(((-1*(dOne**2))/2)))*math.exp(((-1*MTE)*Dividend)))/((Spot*CallIV)*math.sqrt(MTE)))
                CallDelta = ndOne * math.exp(-Dividend * MTE)
                CallTheta = ((((-1*((((Spot*((1/math.sqrt((2*math.pi)))*math.exp(((-1*(dOne**2))/2))))*CallIV)*math.exp(((-1*MTE)*Dividend)))/(2*math.sqrt(MTE))))+((Dividend*Spot)*CallDelta))-(((Rate*Strike)*math.exp(((-1*Rate)*MTE)))*NormDist(dTwo))))/365
            return CallIV, CallDelta, CallGamma, CallTheta, CallVega
                        
        elif Option_Type == 'PE':
            NormDist = norm.cdf
            Rate = 0.065
            Dividend = 0.015
            Upper = 5
            Lower = 0
            TargetPE = OPT
            if MTE==0:
                PutIV = 5
                PutDelta = -1 if Spot < Strike else 0
                PutGamma = 0
                PutTheta = 0
                PutVega = 0
            else:
                while ((Upper - Lower) > 0.00001):
                    dOne = (math.log(Spot/Strike)+((Rate - Dividend + (0.5*(((Upper + Lower)/2)**2)))*MTE))/(((Upper + Lower)/2)*MTE**0.5)
                    dTwo = dOne - ((Upper + Lower)/2) * MTE ** 0.5
                    ndOne = NormDist(dOne)
                    ndTwo = NormDist(dTwo)
                    PutPremium =  math.exp(-Rate * MTE) * Strike * (NormDist(-dTwo)) -math.exp(-Dividend * MTE) *  Spot* (NormDist( -dOne))
                    if(PutPremium > TargetPE):
                        Upper = (Upper + Lower) / 2
                    else:
                        Lower = (Upper + Lower) / 2
                PutIV = (Upper + Lower) / 2
                ndashOne = 1 / (2*math.pi) ** 0.5 * (math.exp(-(dOne**2)/2))
                PutVega = Spot*((MTE**(1/2))*ndashOne*math.exp(-Dividend*MTE))/100
                PutGamma = ((((1/math.sqrt((2*math.pi)))*math.exp(((-1*(dOne**2))/2)))*math.exp(((-1*MTE)*Dividend)))/((Spot*PutIV)*math.sqrt(MTE)))
                PutTheta = (((((-1*((((Spot*((1/math.sqrt((2*math.pi)))*math.exp(((-1*dOne**2)/2))))*PutIV)*math.exp(((-1*MTE)*Dividend)))))/(2*math.sqrt(MTE)))-(((Dividend*Spot)*NormDist((-1*dOne)))*math.exp(((-1*MTE)*Dividend))))+(((Rate*Strike)*math.exp(((-1*Rate)*MTE)))*NormDist((-1*dTwo)))))/365
                PutDelta = (ndOne-1) * math.exp(-Dividend * MTE)
            return PutIV, PutDelta, PutGamma, PutTheta, PutVega
        
    st_g = time.time()
    time1 = datetime.strptime('15:29:59', '%H:%M:%S').time()
    
    if one_file == 'N':
        
        ################## EQUITY DATA
        eq_df = pd.read_csv(eq_file_input_path)
        eq_df = eq_df.rename(columns={'Open' : 'EQ_Open','High' : 'EQ_High','Low' : 'EQ_Low','Close' : 'EQ_Close'})
        eq_df = eq_df[['Date', 'EQ_Open', 'EQ_High', 'EQ_Low', 'EQ_Close']]
        eq_df['Date'] = pd.to_datetime(eq_df['Date'], format = 'mixed',dayfirst=True)
            
        ################## OPTIONS DATA
        opt_df = pd.read_csv(opt_file_input_path)
        opt_df = opt_df.rename(columns={'date' : 'Date','ticker' : 'Ticker','open' : 'Open','high' : 'High','low' : 'Low','close' : 'Close','volume' : 'Volume','Open Int' : 'OpenInterest'})
        opt_df = opt_df.rename(columns={'Open' : 'Adj_Open','High' : 'Adj_High','Low' : 'Adj_Low','Close' : 'Adj_Close'})
        opt_df['Date'] = pd.to_datetime(opt_df['Date'], dayfirst=True , format = 'mixed')
        opt_df['Year'] = opt_df['Date'].dt.year
        opt_df = opt_df.drop(['Year'], axis=1)
        opt_df = opt_df.merge(eq_df, on=['Date'], how='left')
    
    else:
        opt_df = pd.read_csv(file_path)
        
    
    ################# GETTING THE EXPIRY SHEET FROM SQL
    if index != 'FinNifty':
        if index=='BankNifty' and schema[:schema.find('y')+1]=='Weekly':
            engine = pg.connect("dbname='ExpiryDates' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
            exp_df = pd.read_sql(f'select * from "public"."{index}{schema[:schema.find("y")+1]}Expiry"', con=engine)
            engine.close()
        else:
            engine = pg.connect("dbname='ExpiryDates' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
            exp_df = pd.read_sql(f'select * from "public"."{schema[:schema.find("y")+1]}Expiry"', con=engine)
            engine.close()
    elif index == 'FinNifty':
        engine = pg.connect("dbname='ExpiryDates' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
        exp_df = pd.read_sql(f'select * from "public"."{index}{schema[:schema.find("y")+1]}Expiry"', con=engine)
        exp_df.rename(columns={'date':'Date'},inplace=True)
        engine.close()
    exp_df['Date'] = pd.to_datetime(exp_df['Date'], format='mixed',dayfirst=True)
    
    if schema[schema.find('y')+1:] == 'I':
        opt_df = opt_df.merge(exp_df[['Date', 'E1']], on=['Date'], how='left')
        opt_df = opt_df.rename(columns={'E1' : 'Expiry_Date'})
    elif schema[schema.find('y')+1:] == 'II':
        opt_df = opt_df.merge(exp_df[['Date', 'E2']], on=['Date'], how='left')
        opt_df = opt_df.rename(columns={'E2' : 'Expiry_Date'})
    elif schema[schema.find('y')+1:] == 'III':
        opt_df = opt_df.merge(exp_df[['Date', 'E3']], on=['Date'], how='left')
        opt_df = opt_df.rename(columns={'E3' : 'Expiry_Date'})
    
    opt_df = opt_df.rename(columns={'E1' : 'Expiry_Date'})
    
    opt_df['Date'] = pd.to_datetime(opt_df['Date'],format='mixed',dayfirst=True)
    opt_df['Expiry_Date'] = pd.to_datetime(opt_df['Expiry_Date'], format='mixed',dayfirst=True)
    opt_df['YTE'] = (opt_df['Expiry_Date'] - opt_df['Date']).dt.days
    opt_df['YTE/365'] = opt_df['YTE']/365
    opt_df['Final_strike'] = opt_df['Ticker'].str.replace(f'{index.upper()}{schema[:schema.find("y")+1].upper()}-{schema[schema.find("y")+1:]}',"").str[:-2].astype(int)
    # opt_df['Final_strike'] = opt_df['Ticker'].str.replace(f'{stock_name}-I','').str[:-2].astype(float)#.str.replace(f'{schema_name+hyphens[k]}', '').str.replace(f'{index[i].upper()+hyphens[k]}','').str.replace(f'{index[i].upper()+schema_name+hyphens[k]}','').str.replace(f'{index[i].upper()}','')
    opt_df['Option_Type'] = opt_df['Ticker'].str[-2:]
    
    greeks_series = opt_df.apply(lambda x: greeks(x['EQ_Close'], x['Final_strike'], x['YTE/365'], x['Adj_Close'], x['Option_Type']), axis=1)
    #df = df.drop(['Delta'], axis=1)
    try:
        opt_df[['IV', 'Delta', 'Gamma', 'Theta', 'Vega']] = pd.DataFrame(list(greeks_series))
    except:
        print(opt_df)
        print(opt_df)
    opt_df = opt_df.drop(['YTE', 'YTE/365'], axis=1)
    opt_df = opt_df.sort_values(by=['Date', 'Option_Type', 'Final_strike'])
    # df[['IV', 'Delta', 'Gamma', 'Theta', 'Vega']] = pd.DataFrame(list(greeks_series))
    # df = df.sort_values(by=['Date', 'Time', 'Option_Type', 'Final_strike'])
    # df = df[['Date', 'Time', 'Ticker','IV', 'Delta', 'Gamma', 'Theta', 'Vega']]
    opt_df.to_csv(rf'C:\\users\\admin\\desktop\\{index}_{schema}_Greeks.csv',index=False)
    print("\nGreeks Generated!")
    et_g = time.time()
    print("Time taken to generate greeks",et_g-st_g)


Greeks Generated!
Time taken to generate greeks 1.7970552444458008
